<a href="https://colab.research.google.com/github/the-redlord/Image-super-resolution_keras/blob/master/Image_super-resolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading Dataset

In [1]:
!wget -O dataset.zip 'https://d3c33hcgiwev3.cloudfront.net/2Pkn2dIvSzu5J9nSL_s77w_d428a2188ff44626be89908f348634e6_Completed_Notebook_Data_Autoencoders.zip?Expires=1593820800&Signature=U3pa2XSj1lnLKtAv7mALc8VLw1bnuCYqEi1qlByZCvCbPWPUPj8CKkA2oTOP47v22nTh7cFNXamXFN9xO1l0VSiyV4ZIRSu49mJxwmEklJDnCdi4vls0lPG~nLeUQ8g-~5VYc0aSzaeMPJYpQsRvDs2NPMw7hgzkYUb6tTvWqCU_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A'

--2020-07-03 07:21:07--  https://d3c33hcgiwev3.cloudfront.net/2Pkn2dIvSzu5J9nSL_s77w_d428a2188ff44626be89908f348634e6_Completed_Notebook_Data_Autoencoders.zip?Expires=1593820800&Signature=U3pa2XSj1lnLKtAv7mALc8VLw1bnuCYqEi1qlByZCvCbPWPUPj8CKkA2oTOP47v22nTh7cFNXamXFN9xO1l0VSiyV4ZIRSu49mJxwmEklJDnCdi4vls0lPG~nLeUQ8g-~5VYc0aSzaeMPJYpQsRvDs2NPMw7hgzkYUb6tTvWqCU_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A
Resolving d3c33hcgiwev3.cloudfront.net (d3c33hcgiwev3.cloudfront.net)... 13.249.130.93, 13.249.130.13, 13.249.130.118, ...
Connecting to d3c33hcgiwev3.cloudfront.net (d3c33hcgiwev3.cloudfront.net)|13.249.130.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1005471956 (959M) [application/zip]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>] 958.89M  29.5MB/s    in 34s     

2020-07-03 07:21:42 (28.0 MB/s) - ‘dataset.zip’ saved [1005471956/1005471956]



In [2]:
!unzip dataset.zip
!mkdir -p ./data
!mv ./Completed_Notebook_Data_Autoencoders/data/rez/cars_train ./data
!rm -rf ./Completed_Notebook_Data_Autoencoders

Streaming output truncated to the last 5000 lines.
  inflating: Completed_Notebook_Data_Autoencoders/data/rez/cars_train/07795.jpg  
  inflating: Completed_Notebook_Data_Autoencoders/data/rez/cars_train/00199.jpg  
  inflating: Completed_Notebook_Data_Autoencoders/data/rez/cars_train/02603.jpg  
  inflating: Completed_Notebook_Data_Autoencoders/data/rez/cars_train/01410.jpg  
  inflating: Completed_Notebook_Data_Autoencoders/data/rez/cars_train/06009.jpg  
  inflating: Completed_Notebook_Data_Autoencoders/data/rez/cars_train/02445.jpg  
  inflating: Completed_Notebook_Data_Autoencoders/data/rez/cars_train/07253.jpg  
  inflating: Completed_Notebook_Data_Autoencoders/data/rez/cars_train/06771.jpg  
  inflating: Completed_Notebook_Data_Autoencoders/data/rez/cars_train/02813.jpg  
  inflating: Completed_Notebook_Data_Autoencoders/data/rez/cars_train/03573.jpg  
  inflating: Completed_Notebook_Data_Autoencoders/data/rez/cars_train/00866.jpg  
  inflating: Completed_Notebook_Data_Autoencode

# Image Super Resolution using Autoencoders

![image_high_res_low_res](images/high_res_v_low_res.jpg)

## Task 1: Project Overview and Import Libraries

In [3]:
import os
import re
from scipy import ndimage, misc
from skimage.transform import resize, rescale
from matplotlib import pyplot
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)

from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import Conv2DTranspose, UpSampling2D, add
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
import tensorflow as tf
print(tf.__version__)

2.2.0


In [4]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:tensorflow:Initializing the TPU system: grpc://10.50.108.50:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.50.108.50:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU')]


## Task 2: What are Autoencoders?

![image3](images/autoencoder.jpg)

Credit: Autoencoder Schema by <a href="https://blog.keras.io/img/ae/autoencoder_schema.jpg">Francois Chollet, 2016</a>.

![image_schema](images/complete_autoencoder.png)

## Task 3: Build the Encoder

## Task 4: Build the Decoder to Complete the Network

![image4](images/decoder.png)

## Task 5: Create Dataset and Specify Training Routine

In [5]:
def train_batches(just_load_dataset=False):

    batches = 256 

    batch = 0 
    batch_nb = 0 
    max_batches = -1 
    
    ep = 4 

    images = []
    x_train_n = []
    x_train_down = []
    
    x_train_n2 = [] 
    x_train_down2 = []
    
    for root, dirnames, filenames in os.walk("./data/rez/cars_train"):
        for filename in filenames:
            if re.search("\.(jpg|jpeg|JPEG|png|bmp|tiff)$", filename):
                if batch_nb == max_batches: 
                    return x_train_n2, x_train_down2
                filepath = os.path.join(root, filename)
                image = pyplot.imread(filepath)
                if len(image.shape) > 2:
                        
                    image_resized = resize(image, (256, 256))
                    x_train_n.append(image_resized)
                    x_train_down.append(rescale(rescale(image_resized, 0.5), 2.0))
                    batch += 1
                    if batch == batches:
                        batch_nb += 1

                        x_train_n2 = np.array(x_train_n)
                        x_train_down2 = np.array(x_train_down)
                        
                        if just_load_dataset:
                            return x_train_n2, x_train_down2
                        
                        print('Training batch', batch_nb, '(', batches, ')')

                        autoencoder.fit(x_train_down2, x_train_n2,
                            epochs=ep,
                            batch_size=10,
                            shuffle=True,
                            validation_split=0.15)
                    
                        x_train_n = []
                        x_train_down = []
                    
                        batch = 0

    return x_train_n2, x_train_down2

## Task 6: Load the Dataset and Pre-trained Model

## Task 7: Model Predictions and Visualizing the Results